In [0]:
from pyspark.sql.functions import year, month, count, col

df = spark.read.format("delta").load(
    "/Volumes/workspace/default/silver/corte_agua/")

display(df)


In [0]:
col_data = None
for c in df.columns:
    if "data" in c and "corte" in c:
        col_data = c
        break

if col_data is None:
    raise Exception("Nenhuma coluna de data de corte encontrada!")


In [0]:

df_mes = (
    df.groupBy(
        year(col(col_data)).alias("ano"),
        month(col(col_data)).alias("mes")
    )
    .agg(count("*").alias("qtd_cortes"))
    .orderBy("ano", "mes")
)

display(df_mes)


In [0]:
df_mes.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/default/gold/corte_agua/cortes_por_mes/")

print("Gold 01 — Cortes por mês gerado com sucesso!")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold_corte_agua_gold_01 AS
SELECT *
FROM delta.`/Volumes/workspace/default/gold/corte_agua/cortes_por_mes`;
